# Dataset Preparation for Florence-2 fine tuning for object detection

Each image must include both a prefix and a suffix. For fine-tuning Florence-2 on an object detection task, the prefix (prompt) remains constant. The suffix, which is the expected model response, describes each bounding box with the following structure:

{class_name}<loc{x1}><loc{y1}><loc{x2}><loc{y2}>. In this structure, the values x1, y1, x2, and y2 represent the coordinates of the bounding box vertices.

https://huggingface.co/microsoft/Florence-2-large

<img src="florence2_1.jpg">

<img src="florence2_2.jpg">

> We are going to use this images dataset: https://universe.roboflow.com/universiti-malaysia-pahang-qcvas/objectdetection-ngxjp/dataset/5

In [1]:
import os
import json
import sys

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
def yolo_parsing(annotation_file):
    """
    Parses a YOLO annotation file and converts it into a JSON object with image name, prefix, and suffix.

    Args:
        annotation_file (str): Path to the YOLO annotation file.

    Returns:
        dict: A JSON object containing:
            - image (str): The name of the image file corresponding to the annotation.
            - prefix (str): A constant prefix string "<OD>".
            - suffix (str): A concatenated string of bounding box descriptions, each in the format:
                {class_name}<loc_{x1}><loc_{y1}><loc_{x2}><loc_{y2}>.
                The class_name is replaced with a descriptive label, and x1, y1, x2, y2 are the coordinates of the bounding box vertices.
    """
    # opening the file
    with open(annotation_file, 'r') as f:
        lines = f.readlines()

    image_name = os.path.basename(annotation_file).replace('.txt', '.jpg')
    prefix = "<OD>"
    suffix_lines = []

    # Processing
    for line in lines:
        parts = line.strip().split()
        class_name = parts[0]
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])

        x1 = int((x_center - width / 2) * 1000)
        y1 = int((y_center - height / 2) * 1000)
        x2 = int((x_center + width / 2) * 1000)
        y2 = int((y_center + height / 2) * 1000)

        # Replacing class values with class labels
        if class_name == '0':
            class_name = 'DC Power Supply'
        elif class_name == '1':
            class_name = 'Digital Multimeter'
        elif class_name == '2':
            class_name = 'Digital Trainer'
        elif class_name == '3':
            class_name = 'Function Generator'
        elif class_name == '4':
            class_name = 'Oscilloscope'
        suffix_line = f"{class_name}<loc_{x1}><loc_{y1}><loc_{x2}><loc_{y2}>"
        suffix_lines.append(suffix_line)

    # Return json object
    json_obj = {
        "image": image_name,
        "prefix": prefix,
        "suffix": "".join(suffix_lines)
    }

    return json_obj

In [4]:
TRAIN_DIR = "images/train"
VALID_DIR = "images/valid"

## Training

In [5]:
train_annotations_dir = os.path.join(TRAIN_DIR, "labels")
train_json_file = os.path.join(TRAIN_DIR, "images/train_annotations.json")

train_annotations_list = []

for filename in os.listdir(train_annotations_dir):
    if filename.endswith(".txt"):
        annotation_file = os.path.join(train_annotations_dir, filename)
        annotation_obj = yolo_parsing(annotation_file)
        json_string = json.dumps(annotation_obj, separators=(',', ':'))
        train_annotations_list.append(json_string)

In [6]:
len(train_annotations_list)

2379

In [7]:
with open(train_json_file, 'w') as json_file:
    json_file.write("\n".join(train_annotations_list))

print(f"Annotations have been written to {train_json_file}")

Annotations have been written to images/train/images/train_annotations.json


In [8]:
!ls $train_json_file -lh

-rwxrwxrwx 1 root root 374K Jan 14 10:24 images/train/images/train_annotations.json


## Validation

In [9]:
valid_annotations_dir = os.path.join(VALID_DIR, "labels")
valid_json_file = os.path.join(VALID_DIR, "images/val_annotations.json")

valid_annotations_list = []

for filename in os.listdir(valid_annotations_dir):
    if filename.endswith(".txt"):
        annotation_file = os.path.join(valid_annotations_dir, filename)
        annotation_obj = yolo_parsing(annotation_file)
        json_string = json.dumps(annotation_obj, separators=(',', ':'))
        valid_annotations_list.append(json_string)

In [10]:
len(valid_annotations_list)

123

In [11]:
with open(valid_json_file, 'w') as json_file:
    json_file.write("\n".join(valid_annotations_list))

print(f"Annotations have been written to {valid_json_file}")

Annotations have been written to images/valid/images/val_annotations.json


In [12]:
!ls $valid_json_file -lh

-rwxrwxrwx 1 root root 20K Jan 14 10:24 images/valid/images/val_annotations.json
